In [1]:
# Testando API YouTube
import json
import pandas as pd
import seaborn as sns
import numpy as np
import requests
import csv 
from datetime import datetime, timezone

import google_auth_oauthlib.flow
import googleapiclient.discovery
import googleapiclient.errors

In [2]:
# Colocando nossa chave da API do YouTube
import os
from getpass import getpass

api_key = getpass("Insira aqui sua API key da OpenIA:")


In [13]:

# Função para buscar os vídeos em alta no YouTube e armazenar os dados em um DF
def videos_em_alta(n_videos: int, region: str):

    # Definindo os parâmetros que queremos
    params = {
        "part": "snippet",
        "chart": "mostPopular",
        "maxResults": n_videos,  # quantidade de vídeos desejado
        "regionCode": region, # região
        "key": api_key
    }

    # Solicitação à API do YouTube
    url = "https://www.googleapis.com/youtube/v3/videos"
    response = requests.get(url, params=params)
    data = response.json()

    videos = data.get("items",[])

    #caso não há vídeos em alta exibirá a mensagem
    if not videos:
        print("Nenhum vídeo em alta foi encontrado! Ou informações não estão disponíveis")

    # dicionário para salvar as informações
    video_dict = {}

    for video in videos:
        video_id = video["id"]
        video_params = {
            "part": "snippet,statistics",
            "id": video_id,
            "key": api_key
        }

    # Solicitação para obter as informações detalhadas do vídeo
        video_response = requests.get(url, params=video_params)
        video_data = video_response.json()

    # Extraindo as informações 
        video_info = video_data.get("items",[])[0]
        title = video_info["snippet"]["title"]
        views = video_info["statistics"]["viewCount"]
        likes = video_info["statistics"]["likeCount"]
        comments = video_info["statistics"]["commentCount"]
        published_at = video_info["snippet"]["publishedAt"]
        
        link_video = f"https://www.youtube.com/watch?v={video_id}"


    # Armazenando as infos no dicionário
        video_dict[title] = {
            "views": views,
            "likes": likes,
            "comentarios": comments,
            "publicacao": published_at,
            "link_video": link_video
        }
    # Salvando os dados em um DataFrame
    df = pd.DataFrame.from_dict(video_dict, orient="index")
    df = df.reset_index().rename(columns={'index': 'video'})

    # Convertendo a coluna 'publicacao' para datetime e criando uma nova coluna com o tempo postado
    current_time = datetime.now(timezone.utc)

    df['publicacao'] = pd.to_datetime(df['publicacao'])
    df["horas-publicacao"] = (current_time - df["publicacao"]).dt.total_seconds() / 3600
    df["horas-publicacao"] = df["horas-publicacao"].round()
    return df


In [4]:
# função para fazer update diário dos novos vídeos
def update_df(df, df_up):
    df = pd.concat([df,df_up])
    return df.reset_index(drop=True)


In [14]:
# Abrindo e salvando o df em uma variável
df = videos_em_alta(10, 'BR')
df

,video,views,likes,comentarios,publicacao,link_video,horas-publicacao
0,[PT/EN] RAMON DINO - Flow #293,1003769,83832,1716,2023-11-05 20:44:55+00:00,https://www.youtube.com/watch?v=BhS9eJrlusU,20.0
1,O QUE AINDA NÃO FALARAM SOBRE A DERROTA DO RAM...,327862,28522,1613,2023-11-05 23:15:00+00:00,https://www.youtube.com/watch?v=GxZ2LaZ7XnA,18.0
2,VITÓRIA X VILA NOVA - AO VIVO E COM IMAGENS - ...,444794,19889,18,2023-11-05 23:26:47+00:00,https://www.youtube.com/watch?v=4hx3SD2qidU,18.0
3,MINHA OPINIÃO SOBRE O ENEM 2023 (Débora Aladim),81944,18212,1238,2023-11-06 02:44:51+00:00,https://www.youtube.com/watch?v=flM1egug_2w,14.0
4,VEJA A DISPUTA DE PÊNALTIS QUE DEU O OURO PRO ...,1477526,41148,1404,2023-11-05 02:23:52+00:00,https://www.youtube.com/watch?v=8IkI6xtW6Nk,39.0
5,Ludmilla canta o Hino Nacional no GP de São Pa...,236677,3218,5386,2023-11-05 17:22:42+00:00,https://www.youtube.com/watch?v=n4dnaeRgiEA,24.0
6,PRÉVIAS RAMON NO PALCO - CLASSIC PHYSIQUE E ME...,2060974,168511,741,2023-11-04 17:08:12+00:00,https://www.youtube.com/watch?v=yARcFmEa-Bg,48.0
7,Race Highlights | 2023 Sao Paulo Grand Prix,4223311,96796,4234,2023-11-05 19:41:38+00:00,https://www.youtube.com/watch?v=WkLTWmlTaJM,22.0
8,Filha de Raulzinho canta e emociona o pai e o ...,1119219,3965,234,2023-11-04 21:53:18+00:00,https://www.youtube.com/watch?v=CmOdW7Bjbac,43.0
9,FORTALEZA 0 X 2 FLAMENGO | MELHORES MOMENTOS ...,809572,33174,1936,2023-11-05 21:13:17+00:00,https://www.youtube.com/watch?v=hDfmhdtaIZw,20.0


In [15]:
df_us = videos_em_alta(10, 'US')


In [16]:

df = update_df(df, df_us)

In [17]:
df

,video,views,likes,comentarios,publicacao,link_video,horas-publicacao
0,[PT/EN] RAMON DINO - Flow #293,1003769,83832,1716,2023-11-05 20:44:55+00:00,https://www.youtube.com/watch?v=BhS9eJrlusU,20.0
1,O QUE AINDA NÃO FALARAM SOBRE A DERROTA DO RAM...,327862,28522,1613,2023-11-05 23:15:00+00:00,https://www.youtube.com/watch?v=GxZ2LaZ7XnA,18.0
2,VITÓRIA X VILA NOVA - AO VIVO E COM IMAGENS - ...,444794,19889,18,2023-11-05 23:26:47+00:00,https://www.youtube.com/watch?v=4hx3SD2qidU,18.0
3,MINHA OPINIÃO SOBRE O ENEM 2023 (Débora Aladim),81944,18212,1238,2023-11-06 02:44:51+00:00,https://www.youtube.com/watch?v=flM1egug_2w,14.0
4,VEJA A DISPUTA DE PÊNALTIS QUE DEU O OURO PRO ...,1477526,41148,1404,2023-11-05 02:23:52+00:00,https://www.youtube.com/watch?v=8IkI6xtW6Nk,39.0
5,Ludmilla canta o Hino Nacional no GP de São Pa...,236677,3218,5386,2023-11-05 17:22:42+00:00,https://www.youtube.com/watch?v=n4dnaeRgiEA,24.0
6,PRÉVIAS RAMON NO PALCO - CLASSIC PHYSIQUE E ME...,2060974,168511,741,2023-11-04 17:08:12+00:00,https://www.youtube.com/watch?v=yARcFmEa-Bg,48.0
7,Race Highlights | 2023 Sao Paulo Grand Prix,4223311,96796,4234,2023-11-05 19:41:38+00:00,https://www.youtube.com/watch?v=WkLTWmlTaJM,22.0
8,Filha de Raulzinho canta e emociona o pai e o ...,1119219,3965,234,2023-11-04 21:53:18+00:00,https://www.youtube.com/watch?v=CmOdW7Bjbac,43.0
9,FORTALEZA 0 X 2 FLAMENGO | MELHORES MOMENTOS ...,809572,33174,1936,2023-11-05 21:13:17+00:00,https://www.youtube.com/watch?v=hDfmhdtaIZw,20.0


In [18]:
# converendo as colunas 'views', 'likes', 'comentarios' para int
df['views'] = df['views'].astype(int)
df['likes'] = df['likes'].astype(int)
df['comentarios'] = df['comentarios'].astype(int)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype              
---  ------            --------------  -----              
 0   video             20 non-null     object             
 1   views             20 non-null     int32              
 2   likes             20 non-null     int32              
 3   comentarios       20 non-null     int32              
 4   publicacao        20 non-null     datetime64[ns, UTC]
 5   link_video        20 non-null     object             
 6   horas-publicacao  20 non-null     float64            
dtypes: datetime64[ns, UTC](1), float64(1), int32(3), object(2)
memory usage: 1008.0+ bytes


In [19]:
# calculando a quantidade de views por hora
df["views/hour"] = (df["views"]/df["horas-publicacao"]).round(2)
df

,video,views,likes,comentarios,publicacao,link_video,horas-publicacao,views/hour
0,[PT/EN] RAMON DINO - Flow #293,1003769,83832,1716,2023-11-05 20:44:55+00:00,https://www.youtube.com/watch?v=BhS9eJrlusU,20.0,50188.45
1,O QUE AINDA NÃO FALARAM SOBRE A DERROTA DO RAM...,327862,28522,1613,2023-11-05 23:15:00+00:00,https://www.youtube.com/watch?v=GxZ2LaZ7XnA,18.0,18214.56
2,VITÓRIA X VILA NOVA - AO VIVO E COM IMAGENS - ...,444794,19889,18,2023-11-05 23:26:47+00:00,https://www.youtube.com/watch?v=4hx3SD2qidU,18.0,24710.78
3,MINHA OPINIÃO SOBRE O ENEM 2023 (Débora Aladim),81944,18212,1238,2023-11-06 02:44:51+00:00,https://www.youtube.com/watch?v=flM1egug_2w,14.0,5853.14
4,VEJA A DISPUTA DE PÊNALTIS QUE DEU O OURO PRO ...,1477526,41148,1404,2023-11-05 02:23:52+00:00,https://www.youtube.com/watch?v=8IkI6xtW6Nk,39.0,37885.28
5,Ludmilla canta o Hino Nacional no GP de São Pa...,236677,3218,5386,2023-11-05 17:22:42+00:00,https://www.youtube.com/watch?v=n4dnaeRgiEA,24.0,9861.54
6,PRÉVIAS RAMON NO PALCO - CLASSIC PHYSIQUE E ME...,2060974,168511,741,2023-11-04 17:08:12+00:00,https://www.youtube.com/watch?v=yARcFmEa-Bg,48.0,42936.96
7,Race Highlights | 2023 Sao Paulo Grand Prix,4223311,96796,4234,2023-11-05 19:41:38+00:00,https://www.youtube.com/watch?v=WkLTWmlTaJM,22.0,191968.68
8,Filha de Raulzinho canta e emociona o pai e o ...,1119219,3965,234,2023-11-04 21:53:18+00:00,https://www.youtube.com/watch?v=CmOdW7Bjbac,43.0,26028.35
9,FORTALEZA 0 X 2 FLAMENGO | MELHORES MOMENTOS ...,809572,33174,1936,2023-11-05 21:13:17+00:00,https://www.youtube.com/watch?v=hDfmhdtaIZw,20.0,40478.60


In [34]:
date = df['publicacao'].dt.date[0]

In [36]:
df.to_csv(f'top_10_youtube_{date}.csv', sep=';',index=False)